# Uso de selectores

Si conocéis el lenguaje CSS entonces ya sabéis lo que vamos a aprender en esta lección.

Los selectores son una técnica para especificar de forma más concreta qué elementos recuperar del árbol:

| Selector                 | Descripción                      | 
|--------------------------|----------------------------------| 
| Etiquetas                | `soup.select("tag")`             | 
| Identificadores          | `soup.select("#id")`             | 
| Clases                   | `soup.select(".class")`          | 
| Atributos                | `soup.select("tag[attr)`         |  
| Etiquetas en etiquetas   | `soup.select("parent child")`    | 

Vamos a scrapear [la página de Python](https://es.wikipedia.org/wiki/Python) en la Wikipedia para poner en práctica algunos de estos selectores:

In [1]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://web.archive.org/web/20220722211457/https://es.wikipedia.org/wiki/Python")
soup = BeautifulSoup(req.text)

title = soup.select("title")[0].getText()
print(title)

Python - Wikipedia, la enciclopedia libre


Haciendo **clic derecho inspeccionar en cualquier navegador** podemos ver el código fuente mientras seleccionamos los elementos de la página.

Por ejemplo, en el primer parágrafo del documento encontramos un resumen del artículo:

In [2]:
resumen = soup.select("p")[0].getText()
print(resumen)

Python es un lenguaje de alto nivel de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código, se utiliza para desarrollar aplicaciones de todo tipo, ejemplos: Instagram, Netflix, Spotify, Panda 3D, entre otros.[2]​ Se trata de un lenguaje de programación multiparadigma, ya que soporta parcialmente la orientación a objetos, programación imperativa y, en menor medida[¿cuál?], programación funcional. Es un lenguaje interpretado, dinámico y multiplataforma.



## Scrapeando el índice

La Wikipedia tiene lo que se conoce como **Tabla de contenidos**, un índice del documento. 

Según el inspector su etiqueta abre con este tag:

```html
<div id="toc" class="toc" role="navigation" aria-labelledby="mw-toc-heading">
```

Es una capa `div` pero tiene un identificador `id` así que podemos usar un selector y luego podemos extraer sus enlaces `a`:

In [3]:
toc = soup.select("#toc")[0]

for a in toc.select("a"):
    print(a.getText())

1 Historia
2 Características y paradigmas
3 Filosofía
4 Modo interactivo
5 Elementos del lenguaje y sintaxis
5.1 Comentarios
5.2 Variables
5.3 Tipos de datos
5.4 Condicionales
5.5 Bucle for
5.6 Bucle while
5.7 Listas y Tuplas
5.8 Diccionarios
5.9 Sentencia Switch Case
5.9.1 Usando if, elif, else
5.9.2 Usando diccionario
5.10 Conjuntos
5.11 Listas por comprensión
5.12 Funciones
5.13 Clases
5.14 Módulos
5.14.1 Instalación de módulos (pip)
5.14.2 Interfaz al sistema operativo
5.14.3 Comodines de archivos
5.14.4 Argumentos de línea de órdenes
5.14.5 Matemática
5.14.6 Fechas y Tiempos
5.14.7 Módulo Turtle
6 Sistema de objetos
7 Biblioteca estándar
8 Implementaciones
9 Incidencias
10 Véase también
11 Referencias
12 Bibliografía
13 Enlaces externos


Con un poco de ingenio y una expresión regular podemos mostrar únicamente los enlaces de primer nivel:

In [4]:
import re

for a in toc.select("a"):
    text = a.getText()
    if re.match(r"\d+ ", text):
        print(text)

1 Historia
2 Características y paradigmas
3 Filosofía
4 Modo interactivo
5 Elementos del lenguaje y sintaxis
6 Sistema de objetos
7 Biblioteca estándar
8 Implementaciones
9 Incidencias
10 Véase también
11 Referencias
12 Bibliografía
13 Enlaces externos


O formatear los de segundo y tercer nivel con espacios:

In [5]:
import re

for a in toc.select("a"):
    text = a.getText()
    if re.match(r"\d+ ", text):
        print(text)
    elif re.match(r"\d+.\d+ ", text):
        print(" ", text)
    elif re.match(r"\d+.\d+.\d+ ", text):
        print("   ", text)

1 Historia
2 Características y paradigmas
3 Filosofía
4 Modo interactivo
5 Elementos del lenguaje y sintaxis
  5.1 Comentarios
  5.2 Variables
  5.3 Tipos de datos
  5.4 Condicionales
  5.5 Bucle for
  5.6 Bucle while
  5.7 Listas y Tuplas
  5.8 Diccionarios
  5.9 Sentencia Switch Case
    5.9.1 Usando if, elif, else
    5.9.2 Usando diccionario
  5.10 Conjuntos
  5.11 Listas por comprensión
  5.12 Funciones
  5.13 Clases
  5.14 Módulos
    5.14.1 Instalación de módulos (pip)
    5.14.2 Interfaz al sistema operativo
    5.14.3 Comodines de archivos
    5.14.4 Argumentos de línea de órdenes
    5.14.5 Matemática
    5.14.6 Fechas y Tiempos
    5.14.7 Módulo Turtle
6 Sistema de objetos
7 Biblioteca estándar
8 Implementaciones
9 Incidencias
10 Véase también
11 Referencias
12 Bibliografía
13 Enlaces externos


Como véis es cuestión de ser creativo y utilizar las herramientas de las que disponemos.

## Scrapeando la caja de información

Otro elemento interesante que encontramos en el artículo es la caja de información arriba a la derecha, si la insepeccionamos veremos que es una tabla con la clase `infobox`: 


```html
<table class="infobox" style="width:22.7em; line-height: 1.4em; text-align:left; padding:.23em;">
```

En el lenguaje HTML las tablas se forman a partir de etiquetas con filas `tr` que contienen cabeceras `th` o celdas con datos `td`.

Podemos empezar recorriendo las filas a ver qué encontramos:

In [6]:
tr_tags = soup.select(".infobox tr")

for tr_tag in tr_tags:
    print(tr_tag.getText())

Python


Desarrollador(es)

Python Software FoundationSitio web oficial
Información general
Extensiones comunes
.py, .pyc, .pyd, .pyo, .pyw, .pyz, .pyi
Paradigma
Multiparadigma: orientado a objetos, imperativo, funcional, reflexivo
Apareció en
1991
Diseñado por
Guido van Rossum
Última versión estable
3.10.5[1]​ (6 de junio de 2022 (1 mes y 6 días))
Sistema de tipos
Fuertemente tipado, dinámico
Implementaciones
CPython, IronPython, Jython, Python for S60, PyPy, ActivePython, Unladen Swallow
Dialectos
Stackless Python, RPython
Influido por
ABC, ALGOL 68, C, Haskell, Icon, Lisp, Modula-3, Perl, Smalltalk, Java
Ha influido a
Boo, Cobra, D, Falcon, Genie, Groovy, Ruby, JavaScript, Cython, Go Latino
Sistema operativo
Multiplataforma
Licencia
Python Software Foundation License
[editar datos en Wikidata]


No está mal pero podríamos reestructurar el contenido de forma que sea más legible.

Como cada `tr` tiene en teoría dos columnas, una con la cabecera `th` a la izquierda y el texto `td` a la derecha, podemos usar los índices para formatear el texto de salida:

In [7]:
tr_tags = soup.select(".infobox tr")

for tr_tag in tr_tags:
    th_tags = tr_tag.select("th")
    td_tags = tr_tag.select("td")
    if len(th_tags) > 0 and len(td_tags) > 0:
        print(f"{th_tags[0].getText().strip()}: {td_tags[0].getText().strip()}")

Extensiones comunes: .py, .pyc, .pyd, .pyo, .pyw, .pyz, .pyi
Paradigma: Multiparadigma: orientado a objetos, imperativo, funcional, reflexivo
Apareció en: 1991
Diseñado por: Guido van Rossum
Última versión estable: 3.10.5[1]​ (6 de junio de 2022 (1 mes y 6 días))
Sistema de tipos: Fuertemente tipado, dinámico
Implementaciones: CPython, IronPython, Jython, Python for S60, PyPy, ActivePython, Unladen Swallow
Dialectos: Stackless Python, RPython
Influido por: ABC, ALGOL 68, C, Haskell, Icon, Lisp, Modula-3, Perl, Smalltalk, Java
Ha influido a: Boo, Cobra, D, Falcon, Genie, Groovy, Ruby, JavaScript, Cython, Go Latino
Sistema operativo: Multiplataforma
Licencia: Python Software Foundation License


## Scrapeando una imagen

Por último nos quedó pendiente el logo del `infobox`, si inspeccionamos la imagen veremos que tiene un tag `img` con muchos atributos:

```html
<img alt="Python-logo-notext.svg" src="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/100px-Python-logo-notext.svg.png" decoding="async" width="100" height="100" srcset="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/150px-Python-logo-notext.svg.png 1.5x, //web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png 2x" data-file-width="110" data-file-height="110">
```

Es un poco difícil recuperar este elemento porque no tiene un identificador único, pero se me ocurre algo mejor...

¿No se encuentra la imagen principal siempre en la tabla `infobox`? Pues vamos a utilizarla para recuperar la primera imagen en su interior:

In [8]:
img = soup.select(".infobox img")[0]

print(img)

<img alt="Python-logo-notext.svg" data-file-height="110" data-file-width="110" decoding="async" height="100" src="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/100px-Python-logo-notext.svg.png" srcset="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/150px-Python-logo-notext.svg.png 1.5x, //web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png 2x" width="100"/>


Genial, solo tenemos que recuperar el enlace de la imagen en su atributo src:

In [9]:
print(img['src'])

//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/100px-Python-logo-notext.svg.png


Utilizando el propio módulo `requests` podemos guardar la imagen en el directorio de este mismo notebook con el nombre que queramos:

In [10]:
# Ponemos el protocolo https: delante porque el enlace no lo incluye
response = requests.get(f"https:{img['src']}")

if response.status_code == 200:
    with open("image.png", 'wb') as f:
        f.write(response.content)

Si no falla en principio es que se ha descargado, podemos mostrar la imagen en markdown con un simple código:

```markdown
![](image.png)
```

![](image.png)

Os recomiendo experimentar por vuestra cuenta para aprender más, en la [documentación oficial](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) tenéis toda la información sobre `bs4` y el manejo de la jerarquía, también tenéis [más ejemplos sobre el uso de select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors).